In [1]:
import json
import jsonschema
import numpy as np
from jsonschema import validate
from qiskit import transpile
from qiskit_aer import AerSimulator
from mitiq import zne, ddd, pec
from mitiq.zne.scaling.folding import fold_global, fold_gates_at_random, fold_all
from mitiq.zne.scaling.layer_scaling import get_layer_folding
from mitiq.zne.scaling.identity_insertion import insert_id_layers
from mitiq.benchmarks import generate_rb_circuits
from arg_function_maps import schema_to_params
from functions_to_use import basic_noise, execute_0s_ideal

# ZNE

<p>
Load the defined ZNE schema and an example of a ZNE experiment (<i>experiment</i>) using the schema-defined structure <br>
  <span style="font-family: monospace;">&nbsp;&nbsp;&nbsp;&nbspzne_schema</span>: defines the shape and requirements of JSON data, in our case the parameters of a zne experiment <br>
  <span style="font-family: monospace;">&nbsp;&nbsp;&nbsp;&nbspzne_experiment</span>: an example of the type of input data we will use the schema to validate, defines a zne experiment
</p>

In [2]:
def load(schema_path):
    with open(schema_path, 'r') as file:
        return json.load(file)

In [3]:
zne_schema =load('./schema/zne_schema.json')
zne_experiment = load('./experiments/zne_experiments/zne_experiment.json')

Both of these objects behave a lot like python dictionaries with several nested dictionaries, and we can examine their contents as such:

In [4]:
print("top-level keys:", zne_schema.keys(), '\n properties keys:',
      zne_schema['properties'].keys(), '\n \n noise_scaling_factors keys and values: \n',
      json.dumps(zne_schema['properties']['noise_scaling_factors'], indent=4), '\n extrapolation keys and values: \n',
      json.dumps(zne_schema['properties']['extrapolation'], indent=4)
)


top-level keys: dict_keys(['$schema', 'title', 'description', 'type', 'properties', 'additionalProperties']) 
 properties keys: dict_keys(['technique', 'noise_scaling_factors', 'noise_scaling_method', 'scale_factor', 'extrapolation']) 
 
 noise_scaling_factors keys and values: 
 {
    "description": "Real scale factors used in the Factory for extrapolating to zero noise case, one for each point that is to be extrapolated",
    "type": "array",
    "items": {
        "type": "number"
    }
} 
 extrapolation keys and values: 
 {
    "description": "Method used to extrapolate the noise scaling factors to points not in the original set",
    "type": "string",
    "enum": [
        "linear",
        "richardson",
        "polynomial",
        "exponential",
        "poly-exp",
        "adaptive-exp"
    ]
}


top-level keys: <br>
<ul>
  <li><span style="font-family: monospace;">$schema</span>: reference to the version of JSON Schema standard being used</li>
  <li><span style="font-family: monospace;">title</span>: readable title for the schema, often for documentation purposes</li>
  <li><span style="font-family: monospace;">description</span>: explanation of the schema's purpose and structure, providing context for users</li>
  <li><span style="font-family: monospace;">type</span>: specifies the data type (object, array, string, etc) of the input data being validated</li>
  <li><span style="font-family: monospace;">properties</span>: defines object that the input data may be expected to have, along with their structure and validation criteria. </li>
    <ul>
      <li>So looking at the at the properties defined in <span style="font-family: monospace;">zne_schema</span>, we see the familiar options that are avaialable when experimentning zne </li>
      <li>Specifically looking at the <i>noise_scaling_factors</i> and <i>extrapolation</i> properties, we can see how different properties can have different expected data types and the 'enum' key can be used to specify a fixed list of options</li>
    </ul>
</ul>

<br>
<br>

The contents of <span style="font-family: monospace;">zne_experiment</span> show an example of what properties may be specified and how:

In [5]:
print(json.dumps(zne_experiment, indent=4))

{
    "noise_scaling_factors": [
        2,
        4,
        6,
        8
    ],
    "noise_scaling_method": "global",
    "extrapolation": "richardson",
    "scale_factor": 2
}


now let's use the schema to validate this experiment (check that it has the necessary structure and parameters defined)

In [6]:
def validate_experiment(experiment, schema):
    try:
        validate(instance=experiment, schema=schema)
        print("validation passed")
    except jsonschema.exceptions.ValidationError as e:
        print("validation failed")
    return None

In [7]:
validate_experiment(zne_experiment, zne_schema)

validation passed


now that the validation has passed, we can extract the implementation parameters from <span style="font-family: monospace;">zne_experiment</span> to actually perform the mitigation

In [8]:
experiment = zne_experiment

In [9]:
# for the sake of this example, we will use an n-qubit randomized benchmarking circuit, which we can generate using mitiq's generate_rb_circuits function
n_qubits = 2
n_cliffords = 2
trials = 1
seed = 22

rb_circ = generate_rb_circuits(n_qubits=n_qubits, num_cliffords=n_cliffords, trials=trials, return_type='qiskit', seed=seed)[0]

print(rb_circ)

     ┌──────────┐   ┌────┐   ┌──────────┐   ┌─────────┐┌────┐┌──────────┐»
q_0: ┤ Ry(-π/2) ├───┤ √X ├───┤ Ry(-π/2) ├─■─┤ Ry(π/2) ├┤ √X ├┤ Ry(-π/2) ├»
     └──┬───┬───┘┌──┴────┴──┐└──────────┘ │ ├─────────┤├───┬┘└┬───────┬─┘»
q_1: ───┤ X ├────┤ Ry(-π/2) ├─────────────■─┤ Ry(π/2) ├┤ Y ├──┤ Rx(0) ├──»
        └───┘    └──────────┘               └─────────┘└───┘  └───────┘  »
«     ┌───────┐   ┌─────────┐     ┌──────┐  ┌──────────┐   ┌────┐  ┌──────────┐»
«q_0: ┤ Rx(0) ├─■─┤ Ry(π/2) ├─■───┤ √Xdg ├──┤ Ry(-π/2) ├───┤ √X ├──┤ Ry(-π/2) ├»
«     └───────┘ │ └─┬──────┬┘ │ ┌─┴──────┴─┐└──┬────┬──┘┌──┴────┴─┐└──────────┘»
«q_1: ──────────■───┤ √Xdg ├──■─┤ Ry(-π/2) ├───┤ √X ├───┤ Ry(π/2) ├────────────»
«                   └──────┘    └──────────┘   └────┘   └─────────┘            »
«     ┌───────┐   ┌─────────┐┌────┐
«q_0: ┤ Rx(0) ├─■─┤ Ry(π/2) ├┤ √X ├
«     └───────┘ │ └──┬───┬──┘├────┤
«q_1: ──────────■────┤ Y ├───┤ √X ├
«                    └───┘   └────┘


In [10]:
# let's calculate the noisless case of executing the circuit. A randomized benchmarking circuit should be equivalent to the identity operation, so we should expect the ideal output to be the all-zero state

ideal_circ = rb_circ.copy()
ideal_circ.measure_all()

ideal_backend = AerSimulator()
ideal_transpiled = transpile(ideal_circ, optimization_level=0, backend=ideal_backend)
result = ideal_backend.run(ideal_transpiled, optimization_level=0, shots=1000).result()

counts = result.get_counts(ideal_transpiled)
print("Counts:", counts)

total_shots = sum(counts.values())
probabilities = {state: count / total_shots for state, count in counts.items()}
print("Probabilities:", probabilities)

print('|00...0> probability:', probabilities['0'*n_qubits], ', error:', np.abs(1.0-probabilities['0'*n_qubits]))


Counts: {'00': 1000}
Probabilities: {'00': 1.0}
|00...0> probability: 1.0 , error: 0.0


In [11]:
circ_noisy = rb_circ.copy()
circ_noisy.measure_all()

noise_model = basic_noise(n_qubits=n_qubits)
noisy_backend = AerSimulator(noise_model=noise_model)
transpiled_noisy = transpile(circ_noisy, optimization_level=0, backend=noisy_backend)

result = noisy_backend.run(transpiled_noisy, optimization_level=0, shots=1000).result()
counts = result.get_counts()
print("Counts:", counts)

total_shots = sum(counts.values())
probabilities = {state: count / total_shots for state, count in counts.items()}

print("Probabilities:", probabilities)
print('|00...0> probability:', probabilities['0'*n_qubits], ', error:', np.abs(1.0-probabilities['0'*n_qubits]))


Counts: {'01': 20, '11': 7, '10': 51, '00': 922}
Probabilities: {'01': 0.02, '11': 0.007, '10': 0.051, '00': 0.922}
|00...0> probability: 0.922 , error: 0.07799999999999996


In [14]:
# simple exector that returns the probability of measuring either |00...0>
execute = execute_0s_ideal

# we can also just use this executor to get the ideal and unmitigated expectation values:

print('ideal EV: ', execute(rb_circ, noise_model=False))
print('unmitigated noisy EV: ', execute(rb_circ))

ideal EV:  1.0
unmitigated noisy EV:  0.937


In [15]:
# this is sort of a band-aid solution to the problem of mapping strings to functions, need to think about how to do this better

extrapolation_map = {
    "linear": zne.inference.LinearFactory(scale_factors=experiment['noise_scaling_factors']),
    "richardson": zne.inference.RichardsonFactory(scale_factors=experiment['noise_scaling_factors']),
    "polynomial": zne.inference.PolyFactory(scale_factors=experiment['noise_scaling_factors'], order=0), # need to add order as an option in the metashcema here ...
    "exponential": zne.inference.ExpFactory(scale_factors=experiment['noise_scaling_factors']),
    "poly-exp": zne.inference.PolyExpFactory(scale_factors=experiment['noise_scaling_factors'], order=0), # .. and here
    "adaptive-exp": zne.inference.AdaExpFactory(scale_factor=experiment['noise_scaling_factors'][0], steps=4, asymptote=None), # need to adjust metaschema here for steps
}

noise_scaling_map = {
    "global": fold_global,
    "local_random": fold_gates_at_random,
    "local_all": fold_all,
    "layer": get_layer_folding,
    "identity_scaling": insert_id_layers
}

In [16]:
mitigated_result = zne.execute_with_zne(circuit=rb_circ, executor=execute, factory=extrapolation_map[zne_experiment['extrapolation']], scale_noise=noise_scaling_map[zne_experiment['noise_scaling_method']])

In [17]:
mitigated_result

1.0719999999999994

now we can look at experiments that use different extrapolation methods to see how thier mitigated results compare

In [19]:
linear_experiment = load('./experiments/zne_experiments/zne_experiment_lin.json')
polynomial_experiment = load('./experiments/zne_experiments/zne_experiment_poly.json')
exponential_experiment = load('./experiments/zne_experiments/zne_experiment_exp.json')

validate_experiment(linear_experiment, zne_schema)
validate_experiment(polynomial_experiment, zne_schema)
validate_experiment(exponential_experiment, zne_schema)

validation passed
validation passed
validation passed


In [22]:
linear_result = zne.execute_with_zne(circuit=rb_circ, executor=execute, factory=extrapolation_map[linear_experiment['extrapolation']], scale_noise=noise_scaling_map[linear_experiment['noise_scaling_method']])
polynomial_result = zne.execute_with_zne(circuit=rb_circ, executor=execute, factory=extrapolation_map[polynomial_experiment['extrapolation']], scale_noise=noise_scaling_map[polynomial_experiment['noise_scaling_method']])
exponential_result = zne.execute_with_zne(circuit=rb_circ, executor=execute, factory=extrapolation_map[exponential_experiment['extrapolation']], scale_noise=noise_scaling_map[exponential_experiment['noise_scaling_method']])

print("Linear result:", linear_result)
print("Polynomial result:", polynomial_result)
print("Exponential result:", exponential_result)
print("Richardson result:", mitigated_result)

Linear result: 0.9664999999999999
Polynomial result: 0.79775
Exponential result: 1.0184815115946293
Richardson result: 1.0719999999999994


# DDD

In [24]:
ddd_schema =load('./schema/ddd_schema.json')
ddd_experiment = load('./experiments/ddd_experiments/ddd_experiment.json')

validate_experiment(ddd_experiment, ddd_schema)

validation passed


In [25]:
rule_map = {
    "xx": ddd.rules.xx,
    "yy": ddd.rules.yy,
    "xyxy": ddd.rules.xyxy, 
    "general": ddd.rules.general_rule, # need to adjust ddd_schema here to better allow for this 
    "repeated": ddd.rules.repeated_rule, # .. and this
    "custom": None, # ... and this
}

In [26]:
mitigated_result = ddd.execute_with_ddd(circuit=rb_circ, executor=execute, rule=rule_map[ddd_experiment['rule']])

In [27]:
print("experiment rule: ", ddd_experiment['rule'])

experiment rule:  xx


In [30]:
yy_experiment = load('./experiments/ddd_experiments/ddd_experiment_yy.json')
xyxy_experiment = load('./experiments/ddd_experiments/ddd_experiment_xyxy.json')

validate_experiment(yy_experiment, ddd_schema)
validate_experiment(xyxy_experiment, ddd_schema)

validation passed
validation passed


In [31]:
yy_result = ddd.execute_with_ddd(circuit=rb_circ, executor=execute, rule=rule_map[yy_experiment['rule']])
xyxy_result = ddd.execute_with_ddd(circuit=rb_circ, executor=execute, rule=rule_map[xyxy_experiment['rule']])

In [32]:
print("xx result:", mitigated_result)
print("yy result:", yy_result)
print("xyxy result:", xyxy_result)

xx result: 0.945
yy result: 0.948
xyxy result: 0.947


# PEC

In [33]:
from mitiq.benchmarks import generate_rb_circuits
from qiskit_ibm_runtime.fake_provider import FakeJakartaV2 
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

n_qubits = 2
depth_circuit = 100
shots = 10 ** 4

circuit = generate_rb_circuits(n_qubits, depth_circuit,return_type="qiskit")[0]
circuit.measure_all()

def execute_circuit(circuit):
    """Execute the input circuit and return the expectation value of |00..0><00..0|, which ideally should be 1 for randomized benchmarking circuits."""
    noisy_backend = FakeJakartaV2()
    noisy_result = noisy_backend.run(circuit, shots=shots).result()
    noisy_counts = noisy_result.get_counts(circuit)
    noisy_expectation_value = noisy_counts[n_qubits * "0"] / shots
    return noisy_expectation_value

In [34]:
noise_level = 0.01
reps = represent_operations_in_circuit_with_local_depolarizing_noise(rb_circ, noise_level)
print(len(reps))

13


In [35]:
pec_schema = load('./schema/pec_schema.json')
# print(json.dumps(pec_schema['properties'], indent=4))

In [36]:
pec_experiment = {
    "technique": "pec",
    "operation_representations": reps,
    "num_samples": 100
}

validate_experiment(pec_experiment, pec_schema)

validation passed


In [40]:
params = schema_to_params(pec_experiment)
result = pec.execute_with_pec(rb_circ, execute, **params)
print("PEC result:", result)

PEC result: 0.9512032176170859
